In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer,AutoConfig,AutoModel
import torch


d:\anaconda\envs\llm\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
d:\anaconda\envs\llm\lib\site-packages\transformers\utils\hub.py:111: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(


In [2]:
model_name = "../../model/gpt2"
CLLM = AutoModelForCausalLM.from_pretrained(model_name, trust_remote_code=True)
tokenizer= AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
llm = AutoModel.from_pretrained(model_name, trust_remote_code=True)



### CLLM 跟 LLM的区别
- 多了一个MLPHead,维度为(outputsize,vocal_size)

### gpt2属于自回归模型
- 对于输入x,得到$y_1$,然后将[x,$y_1$]作为输入,接着得到$y_2$
### greed_search vs. beam_search
- greed_search:贪心策略，对于每次的输入y,总是选择概率最大的那个,弊端：输出缺乏多样性,场景：数学(要求精确，而不是多样)
- beam_search:束搜索，输入的时候进行束(k)展开，每次输出都进行k展开，最后计算得到最大概率的输出

### 对logits
- 对任意句子(i have a )，进行tokenizer编码后(假设不包含special_token)，得到input_ids,长度为tokens_len
- 将input_ids输入到model中,即output = model(input_ids=input_ids),
- 得到output.logits,维度为(batch_size,tokens_len,vocab_size)
- 对于非最后一个token,即(batch_size,tokens_len[i],vocab_size),i!=-1,输出的含义:model更加偏向对该位置token的预测,
    - 例如(batch_size,tokens_len[0],vocab_size),是指model对第一个token(i)的预测概率的一个排序
- 而对于最后一个token,即(batch_size,tokens_len[-1],vocab_size),输出的含义:model更加偏向对下一个token的预测
    - 例如(batch_size,tokens_len[-1],vocab_size),更多的是对apple这个token的预测，而不是a这个token的预测

### 函数
- torch.argsort(),返回张量排序后的索引（indices）
    - 例如：lst = [3.1,-0.8,2.5],torch.argsort(lst),得到[1,2,0](升序顺序)
    - torch.argsort(lst,descending=True),得到[0,2,1](降序顺序)
- torch.argmax(),返回张量顺序后最大的索引
    - torch.argmax(lst),得到[0]
- model.generate()
    - max_length:prompt+generation的长度
    - max_new_tokens:generation的长度
    - 默认greed_search,如果beam_num不设置的话
  
### 如何理解tokenizer.decode()的时候，是tokenizer.decode(torch.argmax(lst)),是decode的索引,而不是logits
- 对于model(input_ids = input_ids).logits[:,-1,:],得到的是模型对于vocab再该位置输出token的概率
- 其中logits[:,-1,:]潜在的包含了顺序的关系，可以理解为logits[:,-1,:][0],表示的是对于词汇表token ids为0对应的token的概率
- 然后经过torch.argmax(lst)得到最大概率的索引,假设为100,那么logits[:,-1,:][100],是最大的概率对应的logits,那么根据上面可以得到logits[:,-1,:][100]表示的是对于词汇表第token ids为1000对应的token的概率
- 然后经过tokenizer.decode(torch.argmax(lst)),即tokenizer.decode([100]),就是相应的token

In [3]:
sentence = "Hello, how are you?"

In [ ]:
CLLM.eval()
num_steps = 10
top_k = 5
inputs = tokenizer(sentence, return_tensors="pt")
input_ids = inputs.input_ids
df = {}
for step in range(num_steps):
    with torch.no_grad():
        outputs = CLLM(input_ids=input_ids)
        logits = outputs.logits
        prob = torch.softmax(logits[:, -1, :], dim=-1,dtype=torch.float32)
        sort_prob = torch.sort(prob, dim=-1, descending=True)
        predicted_token_ids = torch.argsort(prob, dim=-1,descending=True)
        dict = {}
        for num in range(predicet_num):
            sing_prob = sort_prob.values[0][num].item() * 100
            dict["percentage"+f"位置{num}"] = tokenizer.decode(predicted_token_ids[0][num])+ "的概率" + f"{sing_prob:.4f}" + "%"
        dict["sentence"+f"第{step}次"] = tokenizer.decode(input_ids[0])
        dd = predicted_token_ids[0][0].unsqueeze(0)
        input_ids = torch.cat([input_ids, dd.unsqueeze(0)], dim=-1) 
        df[f"第{step}次"] = dict


tensor([[-118.5744, -117.4402, -120.4907,  ..., -128.7235, -129.1531,
         -116.4908]])
torch.return_types.sort(
values=tensor([[1.7369e-01, 9.4105e-02, 3.2945e-02,  ..., 2.5862e-14, 2.3738e-14,
         4.7234e-15]]),
indices=tensor([[  198,   314,  1867,  ..., 19476, 33434, 13945]]))
tensor([[  198,   314,  1867,  ..., 19476, 33434, 13945]])
17.369191348552704
9.410453587770462
3.2944701611995697
3.2210029661655426
3.165554627776146
第0次 {'percentage位置0': '\n的概率17.3692%', 'percentage位置1': ' I的概率9.4105%', 'percentage位置2': ' What的概率3.2945%', 'percentage位置3': ' How的概率3.2210%', 'percentage位置4': ' You的概率3.1656%', 'sentence第0次': 'Hello, how are you?'}
tensor([[-183.2317, -177.2168, -180.8680,  ..., -198.4083, -204.2615,
         -179.7054]])
torch.return_types.sort(
values=tensor([[9.9432e-01, 5.6672e-04, 2.9005e-04,  ..., 1.4587e-25, 7.8114e-26,
         6.4557e-26]]),
indices=tensor([[  198,    40,    32,  ...,  7782, 14695, 27013]]))
tensor([[  198,    40,    32,  ...,  7782, 14695, 

In [59]:
df

,第0次,第1次,第2次,第3次,第4次,第5次,第6次,第7次,第8次,第9次
percentage位置0,\n的概率17.3692%,\n的概率99.4316%,I的概率7.2990%,'m的概率41.7102%,a的概率5.9283%,little的概率3.8726%,bit的概率13.4777%,of的概率6.2405%,a的概率81.7259%,nerd的概率2.2674%
percentage位置1,I的概率9.4105%,I的概率0.0567%,"""的概率3.1469%",am的概率12.6050%,not的概率4.2078%,bit的概率1.9160%,nervous的概率5.2475%,nervous的概率4.8432%,an的概率10.6300%,geek的概率1.2833%
percentage位置2,What的概率3.2945%,A的概率0.0290%,You的概率2.3875%,'ve的概率5.5294%,just的概率4.1196%,big的概率1.5891%,tired的概率4.4740%,older的概率4.7552%,both的概率1.7132%,fan的概率1.1876%
percentage位置3,How的概率3.2210%,The的概率0.0254%,The的概率1.9682%,was的概率4.2741%,fine的概率3.7082%,very的概率1.3934%,confused的概率3.7622%,tired的概率4.4886%,everything的概率0.5507%,mystery的概率1.0414%
percentage位置4,You的概率3.1656%,M的概率0.0099%,A的概率1.9201%,have的概率2.7005%,so的概率2.8267%,young的概率1.2249%,surprised的概率2.8281%,confused的概率3.0404%,the的概率0.4627%,mess的概率1.0347%
sentence第0次,"Hello, how are you?",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
sentence第1次,NaN,"Hello, how are you?\n",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
sentence第2次,NaN,NaN,"Hello, how are you?\n\n",NaN,NaN,NaN,NaN,NaN,NaN,NaN
sentence第3次,NaN,NaN,NaN,"Hello, how are you?\n\nI",NaN,NaN,NaN,NaN,NaN,NaN
sentence第4次,NaN,NaN,NaN,NaN,"Hello, how are you?\n\nI'm",NaN,NaN,NaN,NaN,NaN


In [58]:
import pandas as pd
df = pd.DataFrame(df)
